In [ ]:
from irrep.bandstructure import BandStructure

import numpy as np
import scipy
from wannierberri.symmetry.projections import Projection, ProjectionsSet
from wannierberri.w90files import Wannier90data

bands_pw = np.loadtxt("./data/Fe_bands_pw.dat")
bohr_ang = scipy.constants.physical_constants['Bohr radius'][0] / 1e-10
alatt = 5.4235* bohr_ang
bands_pw[:,0]*= 2*np.pi/alatt



bandstructure = BandStructure(code="espresso", 
                              prefix="./data/Fe",
                              normalize=False,
                              irreducible=True,
                              include_TR=True,
                              magmom = [[0,0,1]]
                              )
sg = bandstructure.spacegroup
# sg.show()


positions_atoms = [[0,0,0]] 
proj_s = Projection(position_num=positions_atoms, orbital="s", spacegroup=sg)
proj_p = Projection(position_num=positions_atoms, orbital="p", spacegroup=sg)
proj_d = Projection(position_num=positions_atoms, orbital="d", spacegroup=sg)

projection_sp3d2 = Projection(orbital='sp3d2', position_num=[0, 0, 0], spacegroup=sg)
projection_t2g = Projection(orbital='t2g', position_num=[0, 0, 0], spacegroup=sg)

projections = ProjectionsSet(projections=[projection_sp3d2, projection_t2g])
# projections = ProjectionsSet([proj_s, proj_p, proj_d])


w90data = Wannier90data().from_bandstructure(bandstructure, 
                                             projections=projections, 
                                             files=["mmn",  "amn", "eig", "symmetrizer"], 
                                             normalize=False,
                                             irreducible=True,
                                             read_npz_list=[],
                                             write_npz_list=[])


froz_max=19
# w90data.select_bands(win_min=-np.inf, win_max = 100)

w90data.wannierise(
                init = "amn",
                froz_min=-np.inf,
                froz_max=froz_max,
                print_progress_every=10,
                num_iter=101,
                conv_tol=1e-6,
                mix_ratio_z=1.0,
                sitesym=True,
                )


In [ ]:
from matplotlib import pyplot as plt
from wannierberri.grid import Path
from wannierberri import evaluate_k_path
from wannierberri.system import System_w90
system = System_w90(w90data=w90data)

path = Path(system=system,
            nodes = [[0.0000, 0.0000, 0.0000 ],  #  G 
                     [0.500 ,-0.5000, -0.5000],  #  H 
                     [0.7500, 0.2500, -0.2500],  #  P 
                     [0.5000, 0.0000, -0.5000],  #  N 
                     [0.0000, 0.0000, 0.000  ]],  #  G 
            labels = ["$\Gamma$", "H", "P", "N", "$\Gamma$"],
            length=500)

bands_wb = evaluate_k_path(system=system, path=path)

plt.scatter(bands_pw[:,0], bands_pw[:,1], c="black", s=5)



bands_wb.plot_path_fat(path,
                quantity=None,
                # save_file="Fe_bands.pdf",
                Eshift=0,
                Emin=0, Emax=30,
                iband=None,
                mode="fatband",
                fatfactor=20,
                cut_k=False,
                linecolor="red",
                close_fig=False,
                show_fig=False,
                label=f"WB"
                )

# plt.ylim(-10, 40)
plt.hlines(froz_max, 0, bands_pw[-1,0], linestyles="dashed", label="frozen window", color="black")
plt.legend()
plt.show()